In [7]:
from scrapy import signals
from scrapy.utils.project import get_project_settings
from scrapy.crawler import CrawlerProcess
from scrapy.xlib.pydispatch import dispatcher
from multiprocessing.queues import Queue
import scrapy
import multiprocessing

In [8]:
class CrawlerWorker(multiprocessing.Process):

    def __init__(self, spider, result_queue):
        multiprocessing.Process.__init__(self)
        self.result_queue = result_queue

        self.crawler = CrawlerProcess(get_project_settings())
        #if not hasattr(project, 'crawler'):
        #    self.crawler.install()
        #self.crawler.configure()

        self.items = []
        self.spider = spider
        dispatcher.connect(self._item_passed, signals.item_passed)

    def _item_passed(self, item):
        self.items.append(item)

    def run(self):
        self.crawler.crawl(self.spider)
        self.crawler.start()
        self.crawler.stop()
        self.result_queue.put(self.items)

In [9]:
class FreeozLinkSpider(scrapy.Spider):
    name = "FreeozLinkSpider"
    allowed_domains = ["www.freeoz.org"]
    start_urls = ['http://www.freeoz.org/ibbs/forum-7068-3.html']

    def parse(self, response):
        th_tags=response.xpath('//th[@class="common"]')
        for th_tag in th_tags:
            location=th_tag.xpath('em/text()').extract()[0]
            link=th_tag.xpath('a[contains(@href,"www.freeoz.org") and @onclick="atarget(this)"]/@href').extract()[0]
            print("%s:%s"%(location,link))

In [10]:
def main():
    result_queue = Queue()
    crawler = CrawlerWorker(FreeozLinkSpider(), result_queue)
    crawler.start()
    for item in result_queue.get():
        #print datetime.datetime.now(),item
        print item

In [11]:
if __name__ == '__main__':
    main()

2017-01-23 12:57:06 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 12:57:06 [scrapy] INFO: Scrapy 1.1.1 started (bot: scrapybot)
2017-01-23 12:57:06 [scrapy] INFO: Overridden settings: {}
2017-01-23 12:57:06 [scrapy] INFO: Overridden settings: {}
2017-01-23 12:57:06 [scrapy] INFO: Enabled extensions:
['scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2017-01-23 12:57:06 [scrapy] INFO: Enabled extensions:
['scrapy.extensions.logstats.LogStats',
 'scrapy.extensions.telnet.TelnetConsole',
 'scrapy.extensions.corestats.CoreStats']
2017-01-23 12:57:06 [scrapy] INFO: Enabled downloader middlewares:
['scrapy.downloadermiddlewares.httpauth.HttpAuthMiddleware',
 'scrapy.downloadermiddlewares.downloadtimeout.DownloadTimeoutMiddleware',
 'scrapy.downloadermiddlewares.useragent.UserAgentMiddleware',
 'scrapy.downloadermiddlewares.retry.RetryMiddleware',
 'scrapy.downloadermiddlewares.defaultheaders.DefaultHe